In [11]:
"""
python model_run.py VCN_c09 --type Bushy --modeltype II --protocol runANSingles -H --model XM13nacncoop  -r 5 --sgcmodel cochlea -S MS  -f 16000. --soma-autoinflate --dendrite-autoinflate --noparallel
# python model_run.py VCN_c09 --type Bushy --modeltype II --protocol runANSingles -H --model XM13nacncoop --inputpattern VCN_c10 -r 5 --sgcmodel cochlea -S MS  -f 16000. 
"""
%matplotlib qt
import matplotlib
matplotlib.use('Qt4Agg')
import pickle
import numpy as np
from cnmodel.util import sound
import matplotlib.pyplot as mpl
from pathlib import Path
import pylibrary.PlotHelpers as PH

nreps = 5
# fnb = 'VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=self'
# fnb + '_XM13nacncoop_II_soma=1.489_dend=1.236_Syn000_multisite_001_tonepip_030dB_16000.0_HS.p'
fnb = 'VCN_Cells/VCN_c09/Simulations/AN/AN_Result_VCN_c09_inp=VCN_c10'
fnb += '_XM13nacncoop_II_soma=1.489_dend=1.236_Syn000_multisite_001_tonepip_030dB_16000.0_HS.p'

fnsyn = Path(fnb[:fnb.find('_Syn000') + 4])
fns = fnsyn.parent.glob(fnsyn.name+'*.p')
fnsl0 = sorted(list(fns))
fnsl = []
for i in range(len(fnsl0)):
    if str(fnsl0[i]).find(f'_{nreps:03d}_tonepip') > 0 and str(fnsl0[i]).find(f'_16000.0') > 0:
        fnsl.append(fnsl0[i])
nax = len(fnsl)
for f in fnsl:
    print(str(f.name))

def get_one(fn):
    d = pickle.load(open(fn, 'rb'))
#     print(d.keys())
#     print('NTrials: ', len(d['trials']))
#     print(d['Params'].keys())
    return d


AN_Result_VCN_c09_inp=VCN_c10_XM13nacncoop_II_soma=1.489_dend=1.236_Syn000_multisite_005_tonepip_030dB_16000.0_HS.p
AN_Result_VCN_c09_inp=VCN_c10_XM13nacncoop_II_soma=1.489_dend=1.236_Syn001_multisite_005_tonepip_030dB_16000.0_HS.p
AN_Result_VCN_c09_inp=VCN_c10_XM13nacncoop_II_soma=1.489_dend=1.236_Syn002_multisite_005_tonepip_030dB_16000.0_HS.p
AN_Result_VCN_c09_inp=VCN_c10_XM13nacncoop_II_soma=1.489_dend=1.236_Syn003_multisite_005_tonepip_030dB_16000.0_HS.p
AN_Result_VCN_c09_inp=VCN_c10_XM13nacncoop_II_soma=1.489_dend=1.236_Syn004_multisite_005_tonepip_030dB_16000.0_HS.p
AN_Result_VCN_c09_inp=VCN_c10_XM13nacncoop_II_soma=1.489_dend=1.236_Syn005_multisite_005_tonepip_030dB_16000.0_HS.p
AN_Result_VCN_c09_inp=VCN_c10_XM13nacncoop_II_soma=1.489_dend=1.236_Syn006_multisite_005_tonepip_030dB_16000.0_HS.p
AN_Result_VCN_c09_inp=VCN_c10_XM13nacncoop_II_soma=1.489_dend=1.236_Syn007_multisite_005_tonepip_030dB_16000.0_HS.p
AN_Result_VCN_c09_inp=VCN_c10_XM13nacncoop_II_soma=1.489_dend=1.236_Syn0

/Users/pbmanis/anaconda/envs/py3mpl3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import sys


In [12]:
def make_sound(stimInfo):
    if isinstance(stimInfo['pip_start'], list):
        pips = stimInfo['pip_start']
    else:
        pips = [stimInfo['pip_start']]
    if stimInfo['soundtype'] == 'tonepip':
        stim = sound.TonePip(rate=stimInfo['Fs'], duration=stimInfo['run_duration'],
                          f0=stimInfo['F0'], dbspl=stimInfo['dB'],
                          ramp_duration=stimInfo['RF'], pip_duration=stimInfo['pip_duration'],
                          pip_start=pips)
    elif stimInfo['soundtype'] == 'SAM':
        stim = sound.SAMTone(rate=stimInfo['Fs'], duration=stimInfo['run_duration'],
                          f0=stimInfo['F0'], dbspl=stimInfo['dB'],
                          ramp_duration=stimInfo['RF'], 
                          fmod=stimInfo['fmod'], dmod=stimInfo['dmod'],
                          pip_duration=stimInfo['pip_duration'],
                          pip_start=pips)
    else:
        raise ValueError('StimInfo sound type %s not implemented' % stimInfo['soundtype'])

    stimWaveform = stim.generate()
    stimTimebase = stim.time
    return(stimTimebase, stimWaveform)

In [13]:
f, ax = mpl.subplots(nax+1, figsize=(6,10))
# f.figsize=((6,10))
f.suptitle('VCN c09 with VCN c10 inputs')


for i in range(nax):
    if i < nax:
        d = get_one(fnsl[i])
        if d is None:
            continue
    
        for j, t in enumerate(d['trials']['somaVoltage'].keys()):
            ax[i].plot(d['trials']['time'], d['trials']['somaVoltage'][j], linewidth=0.5)
            ax[i].set_ylim([-65., 0.])
            PH.nice_plot(ax[i])
            if i == nax-1:
                PH.calbar(ax[i], [0., -40., 25, 10], unitNames={'x': 'ms', 'y': 'mV'})
            else:
                PH.noaxes(ax[i])
stb, snd = make_sound(d['trials']['stimInfo'])
ax[nax].plot(stb, snd, 'b', linewidth=0.5)
PH.noaxes(ax[nax])
        
#mpl.show()
mpl.show()                    